# 手写数字识别

## 加载相关库

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## 加载相关数据

In [14]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

## 制作训练数据

In [27]:
train_x = train_data.loc[:, 'pixel0':]
train_y = train_data.loc[:, 'label']
test_x = test_data

## 建模

### AlexNet

In [30]:
train_x = tf.constant(train_x, dtype=tf.float32)
train_x = tf.reshape(train_x, [-1, 28, 28, 1])

test_x = tf.constant(test_x, dtype=tf.float32)
test_x = tf.reshape(test_x, [-1, 28, 28, 1])


train_y = tf.one_hot(train_y, depth=10)

tf.Tensor(
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]], shape=(42000, 10), dtype=float32)


In [31]:
train_x, test_x = train_x / 255.0, test_x / 255.0

In [32]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv_1 = layers.Conv2D(32, (3, 3), activation='relu')
        self.conv_2 = layers.Conv2D(64, (3, 3), activation='relu')
        self.conv_3 = layers.Conv2D(64, (3, 3), activation='relu')
        self.linear_1 = layers.Dense(64, activation='relu')
        self.linear_2 = layers.Dense(10)
        self.pooling = layers.MaxPool2D((2, 2))
        self.flatten = layers.Flatten()


    def call(self, input):
        x = self.conv_1(input)
        x = self.pooling(x)
        x = self.conv_2(x)
        x = self.pooling(x)
        x = self.conv_3(x)
        x = self.flatten(x)
        x = self.linear_1(x)
        x = self.linear_2(x)
        return x

In [34]:
model = CNN()
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
loss_function = tf.keras.losses.sparse_categorical_crossentropy

y_pred = model(train_x)

with tf.GradientTape() as tape:
    y_pred = model(train_x)
    print(y_pred.shape, train_y.shape)
    # loss = loss_function(y_pred, train_y)
    # print(loss)


# for i in range(10):
    # with tf.GradientTape() as tape:
    #     y_pred = model(train_x)
    #     print(y_pred)
        # loss = tf.reduce_mean(loss_function(y_pred, train_y))
        # print(loss)
    # grads = tape.gradient(loss, model.variables)
    # optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))



# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

(42000, 10)


Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG13GFamilyCommandBuffer: 0x148cdf6f0>
    label = <none> 
    device = <AGXG13GDevice: 0x13c713000>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x13c6fe600>
        label = <none> 
        device = <AGXG13GDevice: 0x13c713000>
            name = Apple M1 
    retainedReferences = 1
Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG13GFamilyCommandBuffer: 0x148cd87d0>
    label = <none> 
    device = <AGXG13GDevice: 0x13c713000>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x13c6fe600>
        label = <none> 
        device = <AGXG13GDev

tf.Tensor(
[[ 0.02012751  0.11534774  0.02533238 ... -0.17168964 -0.02977245
   0.03512058]
 [ 0.25503397  0.02657911  0.00591695 ... -0.1099663  -0.06091843
   0.07188834]
 [ 0.03720182 -0.01115158 -0.01593997 ... -0.08132021 -0.07337895
   0.0246255 ]
 ...
 [ 0.20608486 -0.0287444  -0.01842406 ... -0.04438282 -0.0446953
   0.00852753]
 [ 0.13667293 -0.00573712  0.01723031 ... -0.0815163  -0.06738288
  -0.01446836]
 [ 0.06192517 -0.0075605   0.000841   ... -0.10714072 -0.01160115
   0.03381626]], shape=(42000, 10), dtype=float32) (42000, 10)


In [9]:
history = model.fit(train_x, train_y, epochs=10)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-08-25 13:57:35.359716: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-25 13:57:35.360128: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


1313/1313 [==============================] - 8s 6ms/step - loss: 0.4168 - accuracy: 0.8692
Epoch 2/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0539 - accuracy: 0.9830
Epoch 3/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0367 - accuracy: 0.9890
Epoch 4/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0272 - accuracy: 0.9915
Epoch 5/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0212 - accuracy: 0.9931
Epoch 6/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0195 - accuracy: 0.9938
Epoch 7/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0177 - accuracy: 0.9945
Epoch 8/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0095 - accuracy: 0.9963
Epoch 9/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0107 - accuracy: 0.9960
Epoch 10/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0090 - accurac

In [28]:
pred = np.argmax(model.predict(test_x), axis=-1).reshape(-1, 1)
pred

array([[2],
       [0],
       [9],
       ...,
       [3],
       [9],
       [2]])

## 输出数据

In [26]:
image_id = np.array(range(1, 28000 + 1)).reshape(-1, 1)
result = pd.DataFrame(data=np.concatenate([image_id, pred], axis=1), columns=['ImageId', 'Label'])

In [27]:
result.to_csv('submission.csv', index=False)